# Task 2 - Uploading Dataset to GCP and Processing the data through E

## Patrick Crouch
## Group 9

In [1]:
from smart_open import open
import json
import pyspark

In [ ]:
sc = pyspark.SparkContext()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/11/20 18:50:09 WARN Utils: Your hostname, MacBook-Pro.local, resolves to a loopback address: 127.0.0.1; using 192.168.68.53 instead (on interface en0)
25/11/20 18:50:09 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/20 18:50:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 52484)
Traceback (most recent call last):
  File "/Users/patri/miniconda3/envs/distributed_computing/lib/python3.13/socketserver.py", line 318, in _handle_request_noblock
    self.process_request(request, client_address)
    ~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/patri/miniconda3/envs/distributed_computing/lib/python3.13/socketserver.py", line 349, in process_request
    self.finish_request(request, client_address)
    ~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/patri/miniconda3/envs/distributed_computing/lib/python3.13/socketserver.py", line 362, in finish_request
    self.RequestHandlerClass(request, client_address, self)
    ~~~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/patri/miniconda3/envs/distributed_computing/lib/python3.13/socketserver.py", line 766, in __init__
    self.handle()
    ~~~~~~~~~~~^^
  File "/Use

#### Question: Does the length of a review correlate to the rating?

In [3]:
bucket = "msds-694-cohort-14-group9/data"
filename = "Electronics.jsonl"
path = f"gs://{bucket}/{filename}"

first_5000 = []

with open(path, "r") as f:
    for i, line in enumerate(f):
        if i >= 5000:
            break
        first_5000.append(json.loads(line))

/Users/patri/miniconda3/envs/distributed_computing/lib/python3.13/site-packages/google/auth/_default.py:108: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/Users/patri/miniconda3/envs/distributed_computing/lib/python3.13/site-packages/google/auth/_default.py:108: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [4]:
base_rdd = sc.parallelize(first_5000)

In [5]:
average_review_length_by_rating = {}
for i in range(5):
    rating = i+1
    filtered_rdd = base_rdd.filter(lambda x: int(x['rating']) == rating)
    review_lengths = filtered_rdd.map(lambda x: len(x['text']))
    average_review_length = review_lengths.reduce(lambda x, y: x+y)/filtered_rdd.count()

    average_review_length_by_rating[rating] = round(average_review_length, 2)


In [ ]:
average_review_length_by_rating

{1: 313.52, 2: 596.99, 3: 567.52, 4: 598.67, 5: 369.23}

25/11/20 19:43:04 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 132046 ms exceeds timeout 120000 ms
25/11/20 19:43:04 WARN SparkContext: Killing executors is not supported by current scheduler.
25/11/20 19:43:06 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:53)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:342)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:132)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$

It seems that review lengths are shorter for 1 and 5 ratings, and longer for the intermediate ratings. If I were to guess why this discrepancy exists, I would think that people who are rating 2, 3 and 4 are thinking more about their rating that those simply choosing the binary best or worst rating.